In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../..')

In [3]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [4]:
from src.infra import config

path = '../../config/vec.yaml'
opt = config.load_config(path)
opt.path = path

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


## Dataset & dataloader

In [7]:
from pathlib import Path

data_root = Path('/home/knpob/Documents/Hinton/data/shape-corr/FAUST_r')

In [13]:
from src.dataset.shape_cor_fast import PairFaustDatasetFast

dataset = PairFaustDatasetFast(
    data_root=data_root,
    phase='train',
    return_faces=True,
    return_L=True,
    return_mass=True,
    num_evecs=200,
    return_evecs=True,
    return_grad=True,
    return_corr=True,
    return_dist=False,
)
len(dataset)

6400

In [19]:
from src.dataloader.shape_cor_batch import BatchShapePairDataLoader

dataloader = BatchShapePairDataLoader(
    dataset,
    batch_size=8,
    shuffle=False,
    num_workers=0,
)
len(dataloader)

800

In [20]:
for idx, data in enumerate(dataloader):
    if idx == 1:
        break

data['first']['name'], data['second']['name'], 

(['tr_reg_000',
  'tr_reg_000',
  'tr_reg_000',
  'tr_reg_000',
  'tr_reg_000',
  'tr_reg_000',
  'tr_reg_000',
  'tr_reg_000'],
 ['tr_reg_008',
  'tr_reg_009',
  'tr_reg_010',
  'tr_reg_011',
  'tr_reg_012',
  'tr_reg_013',
  'tr_reg_014',
  'tr_reg_015'])

In [49]:
data['first']['corr'], data['second']['corr']

(tensor([[4414, 1129, 1929,  ..., 3617,  173, 2482],
         [4414, 1129, 1929,  ..., 3617,  173, 2482],
         [4414, 1129, 1929,  ..., 3617,  173, 2482],
         ...,
         [4414, 1129, 1929,  ..., 3617,  173, 2482],
         [4414, 1129, 1929,  ..., 3617,  173, 2482],
         [4414, 1129, 1929,  ..., 3617,  173, 2482]]),
 tensor([[4466, 1211, 2037,  ..., 3712,  182, 2703],
         [4414, 1138, 1930,  ..., 3604,  141, 2499],
         [4420, 1137, 1903,  ..., 3627, 2400, 2591],
         ...,
         [4402, 1169, 1929,  ..., 3664,  160, 2617],
         [4306, 1089, 1845,  ..., 3604,  146, 2490],
         [4433, 1149, 1941,  ..., 3656,  180, 2601]]))

In [50]:
data['first']['corr'].max()

tensor(4998)

In [53]:
data['first']['corr']

tensor([[4414, 1129, 1929,  ..., 3617,  173, 2482],
        [4414, 1129, 1929,  ..., 3617,  173, 2482],
        [4414, 1129, 1929,  ..., 3617,  173, 2482],
        ...,
        [4414, 1129, 1929,  ..., 3617,  173, 2482],
        [4414, 1129, 1929,  ..., 3617,  173, 2482],
        [4414, 1129, 1929,  ..., 3617,  173, 2482]])

In [79]:
import torch

corr_x = data['first']['corr']   # [B, V]
corr_y = data['second']['corr']  # [B, V]

# template --> corr_x <-(row-wise corresponding)-> corr_y <-- target
B, V_t = corr_x.shape
batch_idx = torch.arange(B, device=corr_y.device).unsqueeze(1).expand(B, V_t)  # [B, V_t] P.S. V_t is the number of vertices in the template shape
p2p_t = torch.full((B, V_t), -1, dtype=torch.long).to(device=corr_y.device)
p2p_t[batch_idx, corr_y] = corr_x
p2p_t.shape

torch.Size([8, 5000])

In [80]:
V_x = data['first']['verts'].shape[1]
p2p = torch.full((B, V_x), -1, dtype=torch.long).to(device=corr_y.device)
p2p.shape

torch.Size([8, 4999])

In [82]:
p2p[:, V_t] = p2p_t[:, :V_x]

IndexError: index 5000 is out of bounds for dimension 1 with size 4999

In [89]:
from src.utils.fmap import corr2pointmap_vectorized

p2p = corr2pointmap_vectorized(
    corr_x=data['first']['corr'],
    corr_y=data['second']['corr'],
    num_verts_y=max(data['second']['num_verts']),
)
p2p

[autoreload of src.metric.infer_sample failed: Traceback (most recent call last):
  File "/home/knpob/miniconda3/envs/fmap/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "/home/knpob/miniconda3/envs/fmap/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 475, in superreload
    module = reload(module)
  File "/home/knpob/miniconda3/envs/fmap/lib/python3.10/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 619, in _exec
  File "<frozen importlib._bootstrap_external>", line 883, in exec_module
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_removed
  File "/home/knpob/Documents/Humboldt/urssm-replicate/notebook/prototype/../../src/metric/infer_sample.py", line 13, in <module>
    class TextureTransferSample(BaseMetric):
  File "/home/knpob/Documents/Humboldt/urssm-replicate/notebook/proto

tensor([[   0,   28,   96,  ...,   -1,   -1,   -1],
        [   0, 2383,   96,  ...,   -1,   -1,   -1],
        [   0, 2383,    2,  ..., 1949,   -1,   -1],
        ...,
        [  71,   16,    2,  ...,   -1,   -1,   -1],
        [   0,   16,  118,  ..., 1949,   -1,   -1],
        [  70,   28,  118,  ...,   -1,   -1,   -1]])

In [90]:
from src.utils.fmap import pointmap2Pyx_vectorized

Pyx = pointmap2Pyx_vectorized(
    p2p=p2p,
    evecs_x = data['first']['evecs'],
    evecs_y = data['second']['evecs'],
    evecs_trans_x = data['first']['evecs_trans'],
    evecs_trans_y = data['second']['evecs_trans'],
    verts_mask_y=data['second']['verts_mask'],
)

RuntimeError: The size of tensor a (200) must match the size of tensor b (5001) at non-singleton dimension 2

In [100]:
evecs_x = data['first']['evecs']
evecs_y = data['second']['evecs']
evecs_trans_x = data['first']['evecs_trans']
evecs_trans_y = data['second']['evecs_trans']
verts_mask_y=data['second']['verts_mask']

K = evecs_x.shape[-1]
index = torch.clamp(p2p, min=0).unsqueeze(-1).expand(-1, -1, K) # [B, V, K]
permuted_evecs_x = torch.gather(evecs_x, dim=1, index=index) * verts_mask_y.unsqueeze(-1) # permute & mask out invalid rows

In [99]:
p = torch.gather(evecs_x, dim=1, index=index)
p * verts_mask_y

RuntimeError: The size of tensor a (200) must match the size of tensor b (5001) at non-singleton dimension 2

In [98]:
verts_mask_y.shape

torch.Size([8, 5001])